# Using pre-trained NN

In [1]:
import numpy as np
import tensorflow as tf
import keras
from matplotlib import pyplot as plt
from imageio import imread
import pickle

Using TensorFlow backend.


In [2]:
# download vgg16 weights
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten # You may need this :)

In [3]:
img_input = Input(shape=(224, 224, 3), name="input")


conv1_1 = Conv2D(64, 3, padding="same", name="conv1_1", activation="relu")(img_input)
conv1_2 = Conv2D(64, 3, padding="same", name="conv1_2", activation="relu")(conv1_1)
pool1   = MaxPool2D(strides=2, padding="same", name="pool1")(conv1_2)

conv2_1 = Conv2D(128, 3, padding="same", name="conv2_1", activation="relu")(pool1)
conv2_2 = Conv2D(128, 3, padding="same", name="conv2_2", activation="relu")(conv2_1)
pool2   = MaxPool2D(strides=2, padding="same", name="pool2")(conv2_2)

conv3_1 = Conv2D(256, 3, padding="same", name="conv3_1", activation="relu")(pool2)
conv3_2 = Conv2D(256, 3, padding="same", name="conv3_2", activation="relu")(conv3_1)
conv3_3 = Conv2D(256, 3, padding="same", name="conv3_3", activation="relu")(conv3_2)
pool3   = MaxPool2D(strides=2, padding="same", name="pool3")(conv3_3)

conv4_1 = Conv2D(512, 3, padding="same", name="conv4_1", activation="relu")(pool3)
conv4_2 = Conv2D(512, 3, padding="same", name="conv4_2", activation="relu")(conv4_1)
conv4_3 = Conv2D(512, 3, padding="same", name="conv4_3", activation="relu")(conv4_2)
pool4   = MaxPool2D(strides=2, padding="same", name="pool4")(conv4_3)

conv5_1 = Conv2D(512, 3, padding="same", name="conv5_1", activation="relu")(pool4)
conv5_2 = Conv2D(512, 3, padding="same", name="conv5_2", activation="relu")(conv5_1)
conv5_3 = Conv2D(512, 3, padding="same", name="conv5_3", activation="relu")(conv5_2)
pool5   = MaxPool2D(strides=2, padding="same", name="pool5")(conv5_3)

flatten = Flatten()(pool5)

fc6     = Dense(4096, activation="relu", name="fc6")(flatten)
drop1   = Dropout(0.5)

fc7     = Dense(4096, activation="relu", name="fc7")(fc6)
drop1   = Dropout(0.5)

fc8     = Dense(1000, activation="relu", name="fc8")(fc7)

output  = Activation("softmax", name="prob")(fc8)

model = keras.Model(img_input, output)

In [4]:
MEAN_VALUES = np.array([104, 117, 123])
IMAGE_W = 224

def preprocess(img):
    img = img[:,:,::-1].astype(np.float64)
    for i in range(3):
        img[:,:,i] -= MEAN_VALUES[i]
        
    # convert from [w,h,3 to 1,w,h,3]
    return img[None]

def deprocess(img):
    img = img.reshape(img.shape[1:])
    for i in range(3):
        img[:,:, i] += MEAN_VALUES[i]
    return img[:, :, :: -1].astype(np.uint8)

img = (np.random.rand(IMAGE_W, IMAGE_W, 3) * 256).astype(np.uint8)

print(np.linalg.norm(deprocess(preprocess(img)) - img))

0.0


### Deploy the network

In [5]:
# load vgg16 weights
import h5py
with h5py.File("vgg16_weights_tf_dim_ordering_tf_kernels.h5", "r") as f:
    vgg16_weights = {k1: {k2:v2.value for k2, v2 in v1.items()} 
                     for k1, v1 in f.items() if len(v1)>0}

Now we should put the weights into their places:

In [6]:
weightKeys = ['block1_conv1', 'block1_conv2', 'block2_conv1', 'block2_conv2', 
              'block3_conv1', 'block3_conv2', 'block3_conv3', 'block4_conv1', 
              'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2',
              'block5_conv3', 'fc1',  'fc2', 'predictions']

In [7]:
weights = [[vgg16_weights[x][x+"_W_1:0"], vgg16_weights[x][x+"_b_1:0"]] for x in weightKeys]

In [8]:
layers = [1,2,4,5,7,8,9,11,12,13,15,16,17,20,21,22]

In [9]:
for i,w in zip(layers, weights):
    model.layers[i].set_weights(w)

In [10]:
genFeatures = keras.Model(img_input, flatten)

In [11]:
from tqdm import tqdm
from skimage.transform import resize as imresize
import os

In [13]:
X = []
Y = []

#this may be a tedious process. If so, store the results in some pickle and re-use them.
for fname in tqdm(os.listdir('train/')[:10]):
    y = fname.startswith("cat")
    img = imread("train/"+fname)
#     print(img.shape)
    img = preprocess(imresize(img,(IMAGE_W,IMAGE_W)))
    features = genFeatures.predict(img)
    Y.append(y)
    X.append(features)

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\hovhannes.sahakyan\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\hovhannes.sahakyan\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|██████████| 10/10 [00:08<00:00,  1.24it/s]


In [22]:
pickle.dump(X,open("X.pkl",'wb'))
pickle.dump(Y,open("Y.pkl",'wb'))